<a href="https://colab.research.google.com/github/catnipglitch/google-colab-notebooks-catnip/blob/main/openai/OpenAI_ImageEdit_Example01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Image Edit API を使用した画像編集ツール

このノートブックは、OpenAIのImage Edit APIを使用して画像を編集するためのインタラクティブなツールを提供します。

## 機能

* **画像のアップロード:** オリジナル画像とマスク画像（オプション）をアップロードできます。
* **プロンプト入力:** 画像編集のためのプロンプトを入力できます。
* **画像サイズ、品質、背景の選択:** 生成される画像のサイズ、品質、背景を指定できます。
* **画像編集と保存:** アップロードした画像とプロンプトに基づいて画像を編集し、Google Driveに保存できます。
* **APIレスポンスの保存:** APIからのレスポンスはJSONファイルとしてGoogle Driveに保存されます。

## 使用方法

1. OpenAI APIキーを `userdata` に設定します。
2. オリジナル画像とマスク画像（オプション）をアップロードします。
3. 画像編集のためのプロンプトを入力します。
4. 画像サイズ、品質、背景を選択します。
5. "画像を編集して保存" ボタンをクリックして、画像を編集し、Google Driveに保存します。

## 注意

* このノートブックは、OpenAIのImage Edit APIを使用するために、`openai` ライブラリがインストールされている必要があります。
* Google Driveにアクセスするために、Google Colabの認証が必要です。

In [38]:
# @title 初期化 {"run":"auto","vertical-output":true}
#@markdown
%%capture

#!pip install Pillow
!pip install openai


from IPython.display import Image as IPyImage, display
from google.colab import files, userdata, drive
#from PIL import Image as PILImage

import io
import os
import base64
import json
import datetime
from pathlib import Path
from openai import OpenAI
import tempfile
#import ipywidgets as widgets



# 一時ディレクトリを作成
temp_dir = tempfile.mkdtemp()




In [39]:
#@title ### OpenAI API接続の準備をするのだ

api_key = userdata.get('OPENAI_API_KEY')

if not api_key:
    print("エラー: OpenAI APIキーが設定されていません。Colabのシークレットに 'OPENAI_API_KEY' を設定してください。")
else:
    try:
        client = OpenAI(api_key=api_key)
        print("OpenAIクライアントの初期化に成功しました。")
    except Exception as e:
        print(f"エラー: OpenAIクライアントの初期化に失敗しました。APIキーを確認してください: {e}")
        client = None # 初期化失敗時はNoneを設定

OpenAIクライアントの初期化に成功しました。


In [40]:
# @title ###  Googleドライブを一度だけマウントするのだ！ {"run":"auto"}
#@markdown
from google.colab import drive   #冒頭で行っておく
drive.flush_and_unmount()


#@markdown **Google Drive マウントフォルダ**
use_google_drive = True  # @param {"type":"boolean"}
google_drive_mount_path = '/content/drive' # @param {"type":"string","placeholder":"/content/drive"}


if use_google_drive:
    print("Google Drive を利用します")
    drive.mount(google_drive_mount_path, force_remount=True)
else:
    print("Google Drive を利用しません")



print("マウント処理終了")



Google Drive を利用します
Mounted at /content/drive
マウント処理終了


In [41]:
# @title 内部処理：入力ファイル読み込み
def read_image_from_filepath(filepath :str=""):
    """
    指定されたファイルパスの画像をPIL.Imageオブジェクトとして読み込む関数。

    Args:
        filepath (str): 読み込む画像のファイルパス。

    Returns:
        PIL.Image.Image or None: 読み込まれたPIL Imageオブジェクト、
                                  またはファイルの読み込みに失敗した場合はNone。
    """
    if not os.path.exists(filepath):
        print(f"エラー: ファイル '{filepath}' が見つかりません。")
        return None

    try:
        image = IPyImage(filepath)

        print(f"ファイル '{filepath}' から画像を読み込みました。")
        return image
    except Exception as e:
        print(f"ファイル '{filepath}' の読み込みに失敗しました: {e}")
        return None


def upload_file() -> IPyImage:
    uploaded = files.upload()

    #アップロードされたファイルを確認
    if len(uploaded) == 0:
        print("ファイルがアップロードされませんでした。")
    else:
        # アップロードされた最初のファイルの名前を取得
        # もし複数のファイルをアップロードした場合、ここを修正して選択できるようにできます
        uploaded_file_name = next(iter(uploaded))
        print(f"アップロードされたファイル: {uploaded_file_name}")
        image = read_image_from_filepath(uploaded_file_name)
        return image


def upload_file_and_display() -> str | None:
    uploaded = files.upload()

    #アップロードされたファイルを確認
    if len(uploaded) == 0:
        print("ファイルがアップロードされませんでした。")

        return None
    else:
        # アップロードされた最初のファイルの名前を取得
        # もし複数のファイルをアップロードした場合、ここを修正して選択できるようにできます
        uploaded_file_name = next(iter(uploaded))
        print(f"アップロードされたファイル: {uploaded_file_name}")
        image = read_image_from_filepath(uploaded_file_name)

        if image is None:
            print("画像の読み込みに失敗しました。")
        else:
            print("画像が利用可能です。")
            display(IPyImage(filename=uploaded_file_name, width=400))

        return uploaded_file_name





In [42]:
# @title 内部処理：ユニークなファイル名を生成

def generate_unique_filename(output_path: str, total_count: int = 1, current_index: int = 1) -> str:
    """
    指定された出力パスと生成数に基づき、重複しないタイムスタンプベースのファイル名を生成します。

    動作仕様:
    1. 現在のタイムスタンプ (YYYY-MMDD-HHMM-SS 形式) をベース名とします。
    2. total_count が 1 より大きい場合、ファイル名に連番と総数 (`_NNofMM`) を追加します。
    3. 生成されたファイル名 (`.png` および `.json` 拡張子を含む) が output_path に既に存在するかを確認します。
    4. 同名のファイルが存在する場合、ファイル名の末尾に連番サフィックス (`_suffix`) を追加して一意性を確保します。
    5. 最終的に一意性が確認されたファイル名 (拡張子なし) を返します。

    Args:
        output_path (str): ファイルを保存するディレクトリのパス。
        total_count (int): 生成されるファイルの総数 (デフォルトは1)。
        current_index (int): 現在生成中のファイルのインデックス (1からtotal_countまで、デフォルトは1)。

    Returns:
        str: 拡張子を含まない、生成されたユニークなファイル名。
    """
    timestamp = datetime.datetime.now().strftime('%Y-%m%d-%H%M-%S')
    base_filename = timestamp

    # n が 1 以外の場合は連番を追加
    if total_count > 1:
        filename = f"{base_filename}_{current_index:02d}of{total_count:02d}"
    else:
        filename = base_filename

    img_path = os.path.join(output_path, f"{filename}.png")
    json_path = os.path.join(output_path, f"{filename}.json")

#　同名のファイルチェック
    index = 0
    while os.path.exists(img_path) or os.path.exists(json_path):
        index += 1
        # 連番サフィックス付きのファイル名を生成
        filename = f"{base_filename}_{current_index:02d}of{total_count:02d}_{index}" if total_count > 1 else f"{base_filename}_{index}"
        img_path = os.path.join(output_path, f"{filename}.png")
        json_path = os.path.join(output_path, f"{filename}.json")

    return filename

In [43]:
# @title 内部処理：Create Image Edit
def create_image_edit_from_openai(input_image_paths: list[str], output_base_path: str,**kwargs) -> str | None:
    """
    OpenAI Image Edit API を利用して画像を生成し、指定されたパスに保存します。
    生成された画像はColabの出力セルに表示されます。

    動作仕様:
    1. input_image_paths で指定された複数のオリジナル画像ファイルを開きます。
    2. 提供されたプロンプトと画像編集パラメータ (`model`, `size`, `quality`, `n`) を使用して、
       OpenAI Image Edit API (`client.images.edit`) を呼び出します。
       必要に応じてマスク画像も使用できます（現在のコードでは original_image_path のみを使用）。
       （補足：マスク画像を使用する場合、API呼び出し時の kwargs に mask パラメータを追加する必要があります。）
    3. APIからのレスポンスに含まれる各生成画像データ (Base64形式) をデコードします。
    4. デコードされた画像データを `IPython.display.Image` を用いてColabの出力セルに表示します。
    5. 生成された各画像に対し、generate_unique_filename 関数を使用して一意なファイル名を生成します。
    6. 生成された画像 (PNG形式) と、使用した生成パラメータを含むJSONファイルを output_base_path ディレクトリに保存します。
    7. API呼び出しまたは処理中にエラーが発生した場合、エラーメッセージを出力し None を返します。
    8. ファイルは自動的に閉じられます (with open 構文使用を推奨)。

    Args:
        input_image_paths (list[str]): 元となる画像ファイルのパスのリスト。OpenAI Image Edit API に渡されます。
        output_base_path (str): 生成された画像とパラメータを保存するディレクトリのパス。
        prompt (str): 画像編集のためのテキストプロンプト。

    Returns:
        str or None: 処理が成功した場合に保存された画像ファイルのフルパス（最初の画像など）、
                     または生成に失敗した場合は None を返します。
                     （現在の実装では常に None を返します。必要に応じて修正してください。）
    """
    # 画像ファイルを開く (with構文で自動的に閉じられるように変更)
    image_files = []
    try:
        for input_path in input_image_paths:
            if os.path.exists(input_path):
                image_files.append(open(input_path, "rb"))
            else:
                print(f"エラー: 入力ファイル '{input_path}' が見つかりません。このファイルはスキップされます。")

        if not image_files:
            print("エラー: 有効な入力画像ファイルが見つかりませんでした。")
            return None

        kwargs["image"] = image_files

        if client is None:
             print("エラー: OpenAIクライアントが初期化されていません。")
             return None

        result = client.images.edit(**kwargs)

        # レスポンスを確認して画像を表示・保存するのだ！
        # result.data はリストなので、各要素を処理
        for i, image_data in enumerate(result.data):
            # 画像データのデコード
            # Base64デコードに失敗する可能性も考慮してtry-exceptを追加
            try:
                image_bytes = base64.b64decode(image_data.b64_json)
            except (base64.binascii.Error, AttributeError) as e:
                 print(f"エラー: 画像 {i + 1} のBase64デコードに失敗しました: {e}")
                 continue # 次の画像データに進む

            # 画像を表示
            print(f"画像 {i + 1} が生成されたのだ！表示するのだ～！")
            # IPython.display.Image はバイトデータまたはファイルパスを受け取る
            # インポート名を考慮して IPyImage に修正
            display(IPyImage(data=image_bytes, width=400)) # IPython.display.Image は IPyImage と別名でインポートされている前提

            # ユニークなファイル名を生成するのだ
            # generate_unique_filename 関数は output_path, total_count, current_index を引数にとる
            # img_count が画像を生成する数であると仮定し、それを total_count に渡す
            filename = generate_unique_filename(output_base_path, total_count=img_count, current_index=i + 1)

            img_file_path = os.path.join(output_base_path, f"{filename}.png")
            json_file_path = os.path.join(output_base_path, f"{filename}.json")

            # 画像ファイルを保存するのだ (with構文で自動的に閉じられるように変更)
            try:
                with open(img_file_path, 'wb') as img_file:
                    img_file.write(image_bytes)
            except IOError as e:
                 print(f"エラー: 画像ファイル '{img_file_path}' の保存に失敗しました: {e}")
                 # 保存に失敗しても処理を続けるか、ここで終了するかは仕様による
                 continue # 今回は保存に失敗した画像はスキップ

            # 生成パラメータをJSONファイルに保存するのだ (with構文で自動的に閉じられるように変更)
            # APIレスポンスの usage を辞書に変換する処理が必要
            # APIレスポンスの構造を確認し、必要な情報をparamsに格納
            params = {
                "prompt": prompt,
                "size": img_size,
                "quality": img_quality,
                "background": img_background, # img_background_en が定義されていること
                "model": "gpt-image-1", # 使用したモデル
                "generation_time": datetime.datetime.now().isoformat(),
                # usage 情報を含める場合は、APIレスポンスの result.usage を適切に処理して追加
                # 例: if hasattr(result, 'usage') and result.usage: # usage属性が存在しNoneでないか確認
                #         params["usage"] = result.usage.__dict__
            }

            try:
                with open(json_file_path, 'w', encoding='utf-8') as json_file:
                    # JSONシリアライズできないオブジェクトが含まれていないか確認
                    json.dump(params, json_file, ensure_ascii=False, indent=2)
            except (IOError, TypeError) as e:
                 print(f"エラー: パラメータファイル '{json_file_path}' の保存に失敗しました: {e}")
                 # 保存に失敗しても処理を続けるか、ここで終了するかは仕様による
                 pass # パラメータ保存失敗は致命的でないと仮定して続行

            print(f"画像を保存したのだ: {img_file_path}")
            print(f"パラメータも保存したのだ: {json_file_path}")

    except Exception as e: # OpenAI API呼び出し自体で発生する可能性のあるエラーを含む広範な例外捕捉
        # OpenAI APIからの特定のエラーを捕捉するとより良い
        # 例: from openai import APIError
        # except APIError as e:
        #    print(f"OpenAI APIエラー: {e.status_code} - {e.response}")
        # except Exception as e:
        print(f"画像生成処理中に予期せぬエラーが起きてしまいました: {e}")
        return None
    finally:
        # 開いたファイルをすべて閉じる
        for f in image_files:
            f.close()

    # 関数が成功した場合の返り値をここで指定
    # 例: return saved_img_path # 保存した最初の画像のパスなどを返す
    return None # 現在の仕様では明示的な返り値は不要と仮定

# 入力ファイル設定方法は
下記の方法でファイル名指定できるように準備します。

## - Google Driveにアップロードする。
アップロードしてファイルパスを取得します。
## - Colab 一時フォルダ (/content)へアップロードする。
次のブロックでアップロードするか、Colobファイルウインドウの任意のフォルダへアップロードする。


In [ ]:
# @title #  準備： Colaboの一時フォルダ(/content)へアップロードする。
#@markdown *Colab でアップロードするとカレントディレクトリである /content配下に作成されます。*
#@markdown *複数のファイルがある場合はアップロードを繰り返してください。表示は上書きされますが、contentフォルダに残ります）

tmp_colabo_uploaded_filename = upload_file_and_display();

if tmp_colabo_uploaded_filename is None:
    print(f"画像の読み込みに失敗しました。:{tmp_colabo_uploaded_filename}")
else:
    print(f"画像のアップロードできました。ファイルパスを入力ファイル名に入力してください。")
    print(f"{tmp_colabo_uploaded_filename}")



In [152]:
# @title #  ①入力ファイルの指定 {"run":"auto","vertical-output":true}
#@markdown ファイル名のみの場合はカレントディレクトリの **/content配下** のファイルが読まれます

#@markdown **入力ファイル設定**

input_filename_00 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/allec-gomes-xnRg3xDcNnE-unsplash.jpg" # @param {"type":"string"}
input_filename_01 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/bere-del-valle-3-AMvwRq7fY-unsplash.jpg" # @param {"type":"string"}
input_filename_02 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/debby-hudson-8iCWV_mNRhs-unsplash.jpg" # @param {"type":"string"}
input_filename_03 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/fernando-andrade-nAOZCYcLND8-unsplash.jpg" # @param {"type":"string"}
input_filename_04 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/mae-mu-U1iYwZ8Dx7k-unsplash.jpg" # @param {"type":"string"}
input_filename_05 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/mae-mu-vbAEHCrvXZ0-unsplash.jpg" # @param {"type":"string"}
input_filename_06 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/mockup-graphics-F-oW3L-L6rk-unsplash.jpg" # @param {"type":"string"}
input_filename_07 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/mockup-graphics-Kl3467edwsE-unsplash.jpg" # @param {"type":"string"}
input_filename_08 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/quaritsch-photography-3HhXWJzG5Ko-unsplash.jpg" # @param {"type":"string"}
# まだそんなに使わないが。。
input_filename_09 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/mockup-graphics-haSJEJYzl5A-unsplash.jpg" # @param {"type":"string"}
input_filename_10 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/mockup-graphics-jHcKq383ibg-unsplash.jpg" # @param {"type":"string"}
input_filename_11 = "/content/drive/MyDrive/Colab-Sample-Images/fruits/quaritsch-photography-3HhXWJzG5Ko-unsplash.jpg" # @param {"type":"string"}
input_filename_12 = "" # @param {"type":"string"}
input_filename_13 = "" # @param {"type":"string"}
input_filename_14 = "" # @param {"type":"string"}
input_filename_15 = "" # @param {"type":"string"}


# 入力ファイルパスをリストにまとめる
input_filenames = [
    input_filename_00, input_filename_01, input_filename_02, input_filename_03,
    input_filename_04, input_filename_05, input_filename_06, input_filename_07,
    input_filename_08, input_filename_09, input_filename_10, input_filename_11,
    input_filename_12, input_filename_13, input_filename_14, input_filename_15
]

valid_input_filenames = []
invalid_input_filenames = []

# ファイルの存在チェックを行い、リストを分ける
for filename in input_filenames:
    if filename: # 空文字列でないことを確認
        if os.path.exists(filename):
            valid_input_filenames.append(filename)
        else:
            invalid_input_filenames.append(filename)

# 存在しないファイルパスを出力
if invalid_input_filenames:
    print("以下の入力ファイルが見つかりませんでした:")
    for invalid_filename in invalid_input_filenames:
        print(f"- {invalid_filename}")

if not valid_input_filenames:
    print("エラー: 有効な入力ファイルが指定されていません。存在する入力ファイルを指定してください。")
else:
    print("有効な入力ファイルが指定されました。")
    # 後続の処理で valid_input_filenames を使用する
    # 例: print(f"有効な入力ファイルリスト: {valid_input_filenames}")

有効な入力ファイルが指定されました。


In [155]:
# @title ②画像編集プロンプト {"run":"auto"}

#@markdown *プロンプト情報 下記のプロンプトテキストはマージされます*
prompt1 = "入力画像のフルーツを籐のバスケットに入った画像" # @param {type:"string"}
prompt2 = "バスケットは明るいキッチンのテーブルの上に置かれている" # @param {type:"string"}
prompt3 = "絵画の様なアートスタイル" # @param {type:"string"}
prompt4 = "" # @param {type:"string"}

img_size = '1024x1024' # @param ["1024x1024", "1536x1024", "1024x1536"]
img_quality = 'high' # @param ["high", "medium","low"]
img_background = 'opaque' # @param ["transparent", "opaque"]

#@markdown **プロンプト情報 下記のプロンプトテキストはマージされます**
img_count = 4 # @param {"type":"slider","min":1,"max":16,"step":1}


#プロンプトを合成
prompt = prompt1 +","+ prompt2 +","+ prompt3 +"," + prompt4

print(f"prompt : {prompt}")

prompt : 入力画像のフルーツを籐のバスケットに入った画像,バスケットは明るいキッチンのテーブルの上に置かれている,絵画の様なアートスタイル,


In [156]:
# @title ③画像生成の実行


#@markdown ---
#@markdown **出力ファイル設定**
output_fileio = "Google Drive " # @param ["Google Drive ","Colab Temp File"]

output_googledrive_folderpath = '/content/drive/MyDrive/AI_Generated_Images' # @param {type:"string"}
output_Colab_folderpath = '/content/temp_AI_Generated_Images' # @param {type:"string"}

output_subfolder = "fruits" # @param {"type":"string"}


# フォルダが無ければ作るよ
os.makedirs(output_googledrive_folderpath , exist_ok=True)
os.makedirs(output_Colab_folderpath , exist_ok=True)




# フォルダパス処理
# subfolder の先頭と末尾のスラッシュを削除
cleaned_output_subfolder = output_subfolder.strip('/')

#ファイルパスを合成
if output_fileio == "Colab Temp File":
    output_base_path = os.path.join(output_Colab_folderpath, cleaned_output_subfolder)
else:
    output_base_path = os.path.join(output_googledrive_folderpath, cleaned_output_subfolder)


# フォルダが無ければ作るよ
os.makedirs(output_base_path , exist_ok=True)

kwargs = {
    "prompt": prompt,
    "model": "gpt-image-1",  # モデルを指定
    "size": img_size,
    "quality": img_quality,
    "background":img_background,
    "n": img_count,
}

print(f"output_base_path = :{output_base_path}")

# Image Edit ファイルリスト等を渡す
create_image_edit_from_openai(valid_input_filenames,output_base_path,**kwargs)

print("画像生成が完了しました。")


Output hidden; open in https://colab.research.google.com to view.

# Gemini向け資料
## OpenAI Image Edit API - Request Body

このエンドポイントは、1つ以上のソース画像とプロンプトに基づいて、編集または拡張された画像を作成します。gpt-image-1とdall-e-2のみをサポートしています。

### パラメータ

| パラメータ | タイプ | 説明 | 必須 |
|---|---|---|---|
| `image` | string or array | 編集する画像。サポートされている画像ファイルまたは画像の配列である必要があります。gpt-image-1の場合、各画像は25MB未満のpng、webp、またはjpgファイルである必要があります。最大16枚の画像を提供できます。dall-e-2の場合、提供できる画像は1枚だけで、4MB未満の正方形のpngファイルである必要があります。 | **必須** |
| `prompt` | string | 目的の画像のテキストによる説明。dall-e-2の最大長は1000文字、gpt-image-1の最大長は32000文字です。 | **必須** |
| `background` | string or null | 生成される画像の背景の透明度を設定できます。このパラメータは、gpt-image-1でのみサポートされています。transparent、opaque、またはauto（デフォルト値）のいずれかである必要があります。autoを使用すると、モデルは画像に最適な背景を自動的に決定します。transparentの場合、出力形式は透明度をサポートする必要があるため、png（デフォルト値）またはwebpに設定する必要があります。 | オプション |
| `mask` | file | imageが編集される場所を示す、完全に透明な領域（アルファがゼロの場所など）を持つ追加の画像。複数の画像が提供されている場合、マスクは最初の画像に適用されます。有効なPNGファイルで、4MB未満で、imageと同じ寸法である必要があります。 | オプション |
| `model` | string | 画像生成に使用するモデル。dall-e-2とgpt-image-1のみがサポートされています。gpt-image-1に固有のパラメータが使用されない限り、デフォルトはdall-e-2です。 | オプション |
| `n` | integer or null | 生成する画像の数。1から10の間である必要があります。 | オプション |
| `quality` | string or null | 生成される画像の品質。high、medium、lowはgpt-image-1でのみサポートされています。dall-e-2は標準品質のみをサポートしています。デフォルトはautoです。 | オプション |
| `response_format` | string or null | 生成された画像が返される形式。urlまたはb64_jsonのいずれかである必要があります。URLは、画像が生成されてから60分間のみ有効です。gpt-image-1は常にbase64エンコードされた画像を返すため、このパラメータはdall-e-2でのみサポートされています。 | オプション |
| `size` | string or null | 生成される画像のサイズ。gpt-image-1の場合は1024x1024、1536x1024（横長）、1024x1536（縦長）、またはauto（デフォルト値）のいずれか、dall-e-2の場合は256x256、512x512、または1024x1024のいずれかである必要があります。 | オプション |
| `user` | string | エンドユーザーを表す一意の識別子。OpenAIが不正使用を監視および検出するのに役立ちます。 | オプション |


### 戻り値

画像オブジェクトのリストを返します。

Open AI Create Image Edit Example

# Example request
```
# これはコードとして書式設定されます
import base64
from openai import OpenAI
client = OpenAI()

prompt = """
Generate a photorealistic image of a gift basket on a white background
labeled 'Relax & Unwind' with a ribbon and handwriting-like font,
containing all the items in the reference pictures.
"""

result = client.images.edit(
    model="gpt-image-1",
    image=[
        open("body-lotion.png", "rb"),
        open("bath-bomb.png", "rb"),
        open("incense-kit.png", "rb"),
        open("soap.png", "rb"),
    ],
    prompt=prompt
)

image_base64 = result.data[0].b64_json
image_bytes = base64.b64decode(image_base64)

# Save the image to a file
with open("gift-basket.png", "wb") as f:
    f.write(image_bytes)

```

# Response
```
{
  "created": 1713833628,
  "data": [
    {
      "b64_json": "..."
    }
  ],
  "usage": {
    "total_tokens": 100,
    "input_tokens": 50,
    "output_tokens": 50,
    "input_tokens_details": {
      "text_tokens": 10,
      "image_tokens": 40
    }
  }
}
```
